In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cats-and-dogs:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3917177%2F6809272%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240317%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240317T063654Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4d00bf379720e748d0e12d5f63c71a8772a0f3b193bb62d66fd8f672149aea10d1d9cbe567d4f5afda131325176b60c7196e403e16e4920fa55eea44a90cc5adbaab69f98600e965a4e60ba5e83a8e940dbc422fddf5c340229a6ec78e959f7c34aece975562054fcd987c1839c3a0763f2510832aa1354f880787effabf2fc1f041a563e9d5877d0e13b6085c633591d2561f9b481edb82de1b0feac3dae2bbc1471238ff0baaa7dfb3ff50d30dfc90048b86cdaaeabf0a823a0ecd510f0ef238e7823c8a26d29919f11ed302cfd53074a8ab281ed556ef681cef6941468b84b411d02094dcea744a21f58395c3165d54748ddfcdcf5d9418b8602016dd4156'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from PIL import Image
from torchvision import models
from torch.utils.data import ( Dataset, DataLoader,random_split)
from sklearn.model_selection import train_test_split

In [ ]:
model = torchvision.models.googlenet(weights="DEFAULT")

In [ ]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv('/kaggle/input/cats-and-dogs/train.csv')
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [ ]:
in_channel = transforms.Resize(256)
num_classes = 2
learning_rate = 0.00001
num_epochs = 10
batch_size = 64
target_size = (32,32)

In [ ]:
dataset = CatsAndDogsDataset(
    csv_file ='/kaggle/input/cats-and-dogs/train.csv',
    root_dir = '/kaggle/input/cats-and-dogs',
    transform = transforms.Compose([transforms.Resize((32,32)),
                                    transforms.ToTensor(),
                                   ])
)


In [ ]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

In [ ]:
train_dataloader = DataLoader(train_set, batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size, shuffle=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False


In [ ]:
model.fc = nn.Linear(in_features=1024, out_features=num_classes)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
# Train Network
for epoch in range(num_epochs):
    losses = []
    for batch_idx, (data, targets) in enumerate(train_dataloader):
        # Get data to cuda if possible
        data = data
        targets = targets
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or adam step
        optimizer.step()
    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x
            y = y
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    model.train()
print("Checking accuracy on Training Set")
check_accuracy(train_dataloader, model)
print("Checking accuracy on Test Set")
check_accuracy(test_dataloader, model)